# Double Pendulum
Our goal is to solve the double pendulum probem using numerical methods available to us in python

In [21]:
%matplotlib inline
import numpy as np
from scipy.integrate import odeint, solve_ivp

import matplotlib.pyplot as plt
#import modules that are used to solve our equations of motion

In [47]:
#Define a class to do all the heavy lifting for solving the equations
class LagrangianDoublePendulum():
    """
    Pendulum class implements the parameters and Lagrange's equations for 
     a simple pendulum (no driving or damping).
     
    Parameters
    ----------
    L_1 : float
        length of the first simple pendulum
    L_2 : float
        length of the second simple pendulum
    g : float
        gravitational acceleration at the earth's surface

    m_1 : float
        mass of first pendulum
    m_2 : float
        mass of the second pendulum

    Methods
    -------
    dy_dt(t, y)
        Returns the right side of the differential equation in vector y, 
        given time t and the corresponding value of y.
    """
    def __init__(self, L1=1.,L2=1., m1=1.,m2=1., g=1.
                ):
        self.L1 = L1
        self.L2=L2
        self.g = g
    
        self.m1 = m1
        self.m2 = m2
    
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [dphi/dt d^2phi/dt^2]
        
        Parameters
        ----------
        t : float
            time 
        y : float
            A 4-component vector with y[0] = phi(t) and y[1] = dphi/dt
            
        Returns
        -------
        
        """
        
        return [y[1],\
                1/((self.m1+self.m2)*self.L1**2+self.m2*self.L1**2*np.cos(y[0]-y[2]))*\
                (self.m2*self.L1/self.L2*self.g*np.sin(y[2])*np.cos(y[0]-y[2])+self.L1**2*y[1]*y[3]*\
                 np.sin(y[0]-y[2])*np.cos(y[0]-y[2])-2*self.m1*self.g*np.sin(y[0])-self.L1*self.L2*y[1]*y[3]*np.sin(y[0]-y[2])),\
                y[3],\
                1/(self.m2**2/(self.m1+self.m2)*self.L2**2*np.cos(y[0]-y[2])+self.m2*self.L1**2)*\
                (-2*self.m1*self.m2*self.L2/(self.L1(self.m1+self.m2))*self.g*np.cos(y[0]-y[2])*\
                 np.sin(y[2])-self.L2**2*self.m2/(self.m1+self.m2)*\
                 np.sin(y[0]-y[2])/\
                 np.cos(y[0]-y[2])-\
                 self.m2*self.g*np.sin(y[2])+\
                 self.L1*self.L2*y[1]*y[3]*np.sin(y[0]-y[2]))]
    
    def solve_ode(self, t_pts, phi_0, phi_dot_0, 
                  abserr=1.0e-9, relerr=1.0e-9):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [phi_0, phi_dot_0, phi_1, phi_dot_1] 
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        phi, phi_dot = solution.y

        return phi, phi_dot

In [48]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None):
    """
    Generic plotting function: return a figure axis with a plot of y vs. x,
    with line color and style, title, axis labels, and line label
    """
    if ax is None:        # if the axis object doesn't exist, make one
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    # if a label if passed, show the legend
        ax.legend()
    if title is not None:    # set a title if one if passed
        ax.set_title(title)
    if axis_labels is not None:  # set x-axis and y-axis labels if passed  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line

In [49]:
def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

In [50]:
# Labels for individual plot axes
phi_vs_time_labels = (r'$t$', r'$\phi(t)$')
phi_dot_vs_time_labels = (r'$t$', r'$p_\phi$')
state_space_labels = (r'$\phi$', r'$p_\phi$')

# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 50.
delta_t = 0.001

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

L1 = 1.
L2 =1.
g = 1.
m1 = 1.
m2=1.

# Instantiate a pendulum 
p1 = LagrangianDoublePendulum(L1=L1, L2=L2, m1=m1,m2=m2, g=g)


In [51]:
# both plots: same initial conditions
phi_0 = (3./4.)*np.pi
phi_dot_0 = 0.
phi_1 = (3./4.)*np.pi
phi_dot_1 = 1.
phi, phi_dot = p1.solve_ode(t_pts, phi_0, phi_dot_0, phi_1, phi_dot_1)


# start the plot!
fig = plt.figure(figsize=(15,5))
overall_title = 'Simple pendulum from Lagrangian:  ' + \
                rf' $\omega_0 = {p1.omega_0:.2f},$' + \
                rf'  $\phi_0 = {phi_0:.2f},$' + \
                rf' $\dot\phi_0 = {phi_dot_0:.2f}$' + \
                '\n'     # \n means a new line (adds some space here)
fig.suptitle(overall_title, va='baseline')
    
# first plot: phi plot 
ax_a = fig.add_subplot(1,3,1)                  

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(t_pts[start : stop], phi[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='blue',
            label=None, 
            title=r'$\phi(t)$', 
            ax=ax_a)    
                              
# second plot: phi_dot plot 
ax_b = fig.add_subplot(1,3,2)                  

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(t_pts[start : stop], phi_dot[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='blue',
            label=None, 
            title=r'$p_\phi$', 
            ax=ax_b)    

# third plot: state space plot from t=30 to t=50   
ax_c = fig.add_subplot(1,3,3)                  

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(phi[start : stop], phi_dot[start : stop], 
            axis_labels=state_space_labels, 
            color='blue',
            label=None, 
            title='State space', 
            ax=ax_c)    

fig.tight_layout()
fig.savefig('simple_pendulum_Lagrange.png', bbox_inches='tight')  

TypeError: 'float' object is not callable